In [1]:
import os, pickle, sys
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import glob
from prettytable import PrettyTable
from tqdm import tqdm
from torch.utils.data import Dataset

import torch
from torch import nn
from torch.nn import Linear,Sequential
from torch.utils.data import DataLoader
import torch.nn.functional as func
import time
from math import isnan,isinf,log,exp
import random

In [2]:
from nas_201_api import NASBench201API as API
api = API('../data/NAS-Bench-201-v1_0-e61699.pth')
api.verbose = False

try to create the NAS-Bench-201 api from ../data/NAS-Bench-201-v1_0-e61699.pth


In [3]:
torch.set_default_dtype(torch.float64)
class zyt(nn.Module):
    def __init__(self):
        super(zyt,self).__init__()
        self.model1=Sequential(
            nn.LayerNorm(188),
            Linear(188,128),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
            #nn.Dropout(p=0.2),
            Linear(128,64),
#             nn.BatchNorm1d(64),
            nn.ReLU(),
            Linear(64,10),
            nn.ReLU(),
           #nn.Dropout(p=0.5),
            Linear(10,1),             
            nn.Sigmoid()
        )
    
    def forward(self,x):
        x=self.model1(x)
        return x
zyt1=zyt()
zyt1.load_state_dict(torch.load("../my_model/zyt1_woBN_full.pth"))
zyt1=zyt1.cuda()

In [4]:
fname=r'../myp/CIFAR100_tensors.p'
rname='CIFAR100'
# fname=r'../myp/ImageNet_tensors.p'
# rname='ImageNet16-120'
# fname=r'../myp/tensors.p'
# rname='CIFAR10'
runs_cifar100=[]
f = open(fname,'rb')
while(1):
    try:
        runs_cifar100.append(pickle.load(f))
    except EOFError:
        break
f.close()
print(fname, len(runs_cifar100))
x_data_cifar100=[]
y_data_cifar100=[]
for idx in range(len(runs_cifar100)):
    temp=runs_cifar100[idx]['logmeasures']['synflow']
    temp=torch.tensor(temp,dtype=torch.float64)
    x_data_cifar100.append(temp)
    y_data_cifar100.append(runs_cifar100[idx]['testacc'])
x_data_cifar100=torch.tensor([item.cpu().detach().numpy() for item in x_data_cifar100],dtype=torch.float64).cuda()
y_data_cifar100=torch.tensor(y_data_cifar100,dtype=torch.float64)
y_data_cifar100=y_data_cifar100/100

../myp/CIFAR100_tensors.p 15625


C:\Users\Administrator\AppData\Local\Temp\ipykernel_22132\4239828888.py:23: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  x_data_cifar100=torch.tensor([item.cpu().detach().numpy() for item in x_data_cifar100],dtype=torch.float64).cuda()


In [5]:
model_proxy_cifar100=zyt1(x_data_cifar100)
model_proxy_cifar100=torch.reshape(model_proxy_cifar100,(len(model_proxy_cifar100),))
model_proxy_cifar100=model_proxy_cifar100.cpu().detach().numpy()

In [6]:
synflow_proxy=[]
f = open('../results_release/nasbench2/nb2_cf100_seed42_dlrandom_dlinfo1_initwnone_initbnone.p','rb')
while(1):
    try:
        d = pickle.load(f)
        synflow_proxy.append(d['logmeasures']['synflow'])
    except EOFError:
        break
f.close()